In [ ]:
# Work 09
## Variables preparation - curb related, entropy related

In [2]:
# loading libraries
import torch
torch.cuda.empty_cache() 
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image
import pandas as pd
import pyproj
from pathlib import Path
import joblib
import seaborn as sn

In [3]:
# loading written functions
from mtp_function_yl import *

### Variables preparation - curb related

In [ ]:
# For accident locations

In [12]:
# loading data
preda = pd.read_csv('/home/yelhe/script/mt/output/pred_accident_label.csv')
predp = pd.read_csv('/home/yelhe/script/mt/output/pred_pseudo_label.csv')
nla = pd.read_csv('/home/yelhe/script/mt/output/n_accident.txt', header=None)
# setting identifier to each image using location identifier and direction number 
nla.rename(columns = {0:'Nr'}, inplace = True)
nla['Nr'] = nla['Nr'].values.astype(str)
nla['id'] = nla['Nr'].str[-15:-1]
nla['dir'] = nla['Nr'].str[-1]
nla1 = nla[['id', 'dir']]
nla2 = nla1
nla3 = nla1
nla4 = nla1
nla2 = nla2.assign(dir = '2')
nla3 = nla3.assign(dir = '3')
nla4 = nla4.assign(dir = '4')
nlas = pd.concat([nla1, nla2, nla3, nla4])
nlas['Nr'] = nlas['id'] + nlas['dir']
nlas0 = nlas
nlas1 = nlas
nlas2 = nlas
nlas3 = nlas
nlas0 = nlas0.assign(p = '0')
nlas1 = nlas1.assign(p = '1')
nlas2 = nlas2.assign(p = '2')
nlas3 = nlas3.assign(p = '3')
nlass = pd.concat([nlas0, nlas1, nlas2, nlas3])
# merging identifier with predicted label
preda = preda[['Nr', 'p']]
preda = preda.groupby(['Nr', 'p']).size().reset_index().rename(columns={0:'count'})
preda['p'] = preda['p'].values.astype(str)
preda['Nr'] = preda['Nr'].values.astype(str)
preda['id'] = preda['Nr'].str[-15:-1]
preda['dir'] = preda['Nr'].str[-1]
pa = nlass.merge(preda, how = 'left', on = ['id', 'Nr', 'dir', 'p'])
pa = pa.sort_values(by = ['id', 'dir', 'p'], ignore_index = True)
pa = pa.fillna(0)
pac = pa.loc[pa['p'] == '0']
pacc = pac.groupby(['id'])['count'].sum().reset_index().rename(columns = {'count':'sum'})
# pacc.to_csv('/home/yelhe/script/mt/output/predcurb_accident_count_sum.csv', index = False)
# calculating statistical value of curb count considering four directions in each location
paccd = pac.groupby(['id', 'dir'])['count'].sum().reset_index()
# mean number of curbs
pacm = paccd.groupby('id', as_index = True)[['count']].mean()
pacm.rename(columns ={'count':'cmean'}, inplace = True)
pacm = pacm.reset_index()
# minimum number of curbs
pacmin = paccd.groupby('id', as_index = True)[['count']].min()
pacmin.rename(columns = {'count':'cmin'}, inplace = True)
pacmin = pacmin.reset_index()
# maximum number of curbs
pacmax = paccd.groupby('id', as_index = True)[['count']].max()
pacmax.rename(columns = {'count':'cmax'}, inplace = True)
pacmax = pacmax.reset_index()
# sum number of curbs
pac1 = pd.merge(pacm, pacmin, on = ['id'])
pac2 = pd.merge(pacc, pacmax, on = ['id'])
pac_all = pd.merge(pac1, pac2, on = ['id'])
pac_all.rename(columns = {'sum': 'csum'}, inplace = True)
# presence of curbs
pac_all = pac_all.assign(cp = 1)
pac_all.loc[pac_all['cmean'] == 0, 'cp'] = 0
# pac_all.to_csv('/home/yelhe/script/mt/output/accident_predictedcurball.csv', index = False)
# dfencv_all.to_csv('/home/yelhe/script/mt/output/accident_crevall_new.csv', index = False)

In [15]:
# For random pseudo locations

In [43]:
# loading data
predp = predp[['Nr', 'p']]
nlp = pd.read_csv('/home/yelhe/script/mt/output/n_pseudo.txt', header = None)
# setting identifier to each image using location identifier and direction number 
nlp.rename(columns = {0:'Nr'}, inplace = True)
nlp['Nr'] = nlp['Nr'].values.astype(str)
nlp['Nr'] = nlp['Nr'].str.zfill(5)
nlp['id'] = nlp['Nr'].str[-5:-1]
nlp['dir'] = nlp['Nr'].str[-1]
nlp1 = nlp[['id','dir']]
nlp2= nlp1
nlp3= nlp1
nlp4= nlp1
nlp2 = nlp2.assign(dir = '2')
nlp3 = nlp3.assign(dir = '3')
nlp4 = nlp4.assign(dir = '4')
nlps = pd.concat([nlp1, nlp2, nlp3, nlp4])
nlps['Nr'] = nlps['id'] + nlps['dir']
nlps0 = nlps
nlps1 = nlps
nlps2 = nlps
nlps3 = nlps
nlps0 = nlps0.assign(p = '0')
nlps1 = nlps1.assign(p = '1')
nlps2 = nlps2.assign(p = '2')
nlps3 = nlps3.assign(p = '3')
nlpss = pd.concat([nlps0, nlps1, nlps2, nlps3])
# merging identifier with predicted label
predp = predp.groupby(['Nr','p']).size().reset_index().rename(columns={0:'count'})
predp['p'] = predp['p'].values.astype(str)
predp['Nr'] = predp['Nr'].values.astype(str)
predp['Nr'] = predp['Nr'].str.zfill(5)
predp['id'] = predp['Nr'].str[-5:-1]
predp['dir'] = predp['Nr'].str[-1]pp = nlpss.merge(predp, how = 'left', on = ['id', 'Nr', 'dir', 'p'])
pp = pp.sort_values(by = ['id', 'dir', 'p'], ignore_index = True)
pp = pp.fillna(0)
ppc = pp.loc[pp['p'] == '0']
ppcc = ppc.groupby(['id'])['count'].sum().reset_index().rename(columns = {'count':'sum'})
# calculating statistical value of curb count considering four directions in each location
ppccd = ppc.groupby(['id', 'dir'])['count'].sum().reset_index()
# mean number of curbs
ppcm = ppccd.groupby('id', as_index = True)[['count']].mean()
ppcm.rename(columns ={'count':'cmean'}, inplace = True)
ppcm = ppcm.reset_index()
# minimum number of curbs
ppcmin = ppccd.groupby('id', as_index = True)[['count']].min()
ppcmin.rename(columns = {'count':'cmin'}, inplace = True)
ppcmin = ppcmin.reset_index()
# maximum number of curbs
ppcmax = ppccd.groupby('id', as_index = True)[['count']].max()
ppcmax.rename(columns = {'count':'cmax'}, inplace = True)
ppcmax = ppcmax.reset_index()
# sum number of curbs
ppc1 = pd.merge(ppcm, ppcmin, on = ['id'])
ppc2 = pd.merge(ppcc, ppcmax, on = ['id'])
ppc_all = pd.merge(ppc1, ppc2, on = ['id'])
ppc_all.rename(columns = {'sum': 'csum'}, inplace = True)
# presence of curbs
ppc_all = ppc_all.assign(cp = 1)
ppc_all.loc[ppc_all['cmean'] == 0, 'cp'] = 0
# ppc_all.to_csv('/home/yelhe/script/mt/output/pseudo_predictedcurball.csv', index = False)
# ppc_all.to_csv('/home/yelhe/script/mt/output/pseudo_crevall_new.csv')

### Preparation of variables - entropy

In [ ]:
# For accident locations

In [ ]:
# loading data
dfen_a = pd.read_csv('/home/yelhe/script/mt/output/accident_entropy_sum_new.csv')
dfacc = pac_all
# dfacc = pd.read_csv('/home/yelhe/script/mt/output/accident_predictedcurball.csv')
predc_a = pacc
# predc_a = pd.read_csv('/home/yelhe/script/mt/output/predcurb_accident_count_sum.csv')
# calculating statistical value of entropy considering four directions in each location
# mean value of entropy variables
dfenm = dfen_a.groupby('id', as_index = True)[['mew','meg','ie']].mean()
dfenm = dfenm.reset_index()
dfc = pd.merge(dfenm, predc_a, on = ['id'])
dfc.rename(columns = {'sum':'cn'}, inplace = True)
# minimum value of entropy variables
dfenmin = dfen_a.groupby('id', as_index = True)[['mew','meg','ie']].min()
dfenmin.rename(columns = {'mew':'mewmin', 'meg':'megmin', 'ie':'iemin'}, inplace = True)
dfenmin = dfenmin.reset_index()
# maximum value of entropy variables
dfenmax = dfen_a.groupby('id', as_index = True)[['mew','meg','ie']].max()
dfenmax.rename(columns = {'mew':'mewmax', 'meg':'megmax', 'ie':'iemax'}, inplace = True)
dfenmax = dfenmax.reset_index()
# merging curb variables and entropy variables into one single file
dfen_all = pd.merge(dfenmin, dfenmax, on = ['id'])
dfenc_all = pd.merge(dfen_all, dfc, on = ['id'])
dfencv_all = pd.merge(dfenc_all, dfacc, on = ['id'])
dfencv_all = dfencv_all.drop(['cn'], axis = 1)
dfencv_all.to_csv('/home/yelhe/script/mt/output/accident_crevall_new.csv', index = False)

In [ ]:
# For random pseudo locations

In [ ]:
# loading data
predc_r = ppc_all
# predc_r = pd.read_csv('/home/yelhe/script/mt/output/pseudo_predictedcurball.csv')
dfen_r = pd.read_csv('/home/yelhe/script/mt/output/pseudo_entropy_sum_new.csv')
# mean value of entropy variables
dfenm = dfen_r.groupby('id', as_index = True)[['mew','meg','ie']].mean()
dfenm = dfenm.reset_index()
dfc = pd.merge(dfenm, predc_r, on = ['id'])
dfc.rename(columns = {'sum':'cn'}, inplace = True)
# minumum value of entropy variables
dfenmin = dfen_r.groupby('id', as_index = True)[['mew','meg','ie']].min()
dfenmin.rename(columns = {'mew':'mewmin', 'meg':'megmin', 'ie':'iemin'}, inplace = True)
dfenmin = dfenmin.reset_index()
# maximum value of entropy variables
dfenmax = dfen_r.groupby('id', as_index = True)[['mew','meg','ie']].max()
dfenmax.rename(columns = {'mew':'mewmax', 'meg':'megmax', 'ie':'iemax'}, inplace = True)
dfenmax = dfenmax.reset_index()
# merging curb variables and entropy variables into one single file
dfen_all = pd.merge(dfenmin, dfenmax, on = ['id'])
dfenc_all = pd.merge(dfen_all, dfc, on = ['id'])
dfenc_all = dfenc_all.assign(cp = 1)
dfenc_all.loc[dfenc_all['cmean'] == 0, 'cp'] = 0
dfenc_all.to_csv('/home/yelhe/script/mt/output/pseudo_crevall_new.csv', index = False)